# API REQUEST

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from dateutil.parser import parse
import datetime

In [2]:
def f2c(f):
    return round( (f - 32) * (5/9) )


def getDay(val):
    return ' '.join(val.split('<p class="period-name">')[-1].split('</p>')[0].split('<br/>'))

In [3]:
url = 'https://forecast.weather.gov/MapClick.php?x=276&y=148&site=lox&zmx=&zmy=&map_x=276&map_y=148#.YVLh87gzZEY'
response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")
forecast_list = soup.findAll('li', class_ = "forecast-tombstone")

date_current = soup.find('div', id = "current_conditions_detail")
date_current = date_current.findAll('tr')[-1].findAll('td')[-1].text.strip()


contents = {'day_of_week':[],'date':[], 'short_desc':[],'temperature':[],'full_description':[]}

for i in range(len(forecast_list)):

    contents['day_of_week'].append( getDay(str( forecast_list[i].find('p', class_ = "period-name") )) )
    contents['short_desc'].append( forecast_list[i].find('p', class_ = "short-desc").text )

    date_current = parse(date_current) + datetime.timedelta(days=i)
    date_current = date_current.strftime('%d/%m/%Y')
    contents['date'].append(date_current)

    temperature = forecast_list[i].find('p', class_ = "temp").text
    temp_breaks = temperature.split()
    temp_celsuis = f2c(float(temp_breaks[1]))
    contents['temperature'].append( temp_breaks[0]+" "+str(temp_celsuis)+"°C" )
    

    contents['full_description'].append( forecast_list[i].find('img').get('title') )

df = pd.DataFrame(contents)
print(df)

       day_of_week        date                   short_desc temperature  \
0   This Afternoon  28/09/2021                        Sunny  High: 22°C   
1        Tonight    29/09/2021  Partly Cloudythen PatchyFog   Low: 16°C   
2      Wednesday    01/10/2021         Patchy Fogthen Sunny  High: 24°C   
3  Wednesday Night  13/01/2021                        Clear   Low: 14°C   
4       Thursday    17/01/2021                        Sunny  High: 28°C   
5   Thursday Night  22/01/2021                        Clear   Low: 16°C   
6         Friday    28/01/2021                        Sunny  High: 32°C   
7     Friday Night  04/02/2021                 Mostly Clear   Low: 16°C   
8       Saturday    10/04/2021                        Sunny  High: 33°C   

                                    full_description  
0  This Afternoon: Sunny, with a high near 72. We...  
1  Tonight: Patchy fog after midnight.  Otherwise...  
2  Wednesday: Patchy fog before noon.  Otherwise,...  
3  Wednesday Night: Clear, wi

C:\Users\Asus\.conda\envs\strive\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
